# Just to hide things

In [1]:
import pandas as pd
import numpy as np

# Further cleaning

In [ ]:
allowed_values = ['Amber', 'Green', 'Red', 'Amber/red', 'Amber/Green']
result['colour_rating_new'] = result['colour_rating_new'].apply(lambda x: x if x in allowed_values else np.nan)

In [ ]:
result = result.rename(columns={'colour_rating_new': 'colour_rating'})

In [ ]:
result['yearly_forecast_new'] = result['yearly_forecast'].combine_first(result['forecast_incl_NGC'])

In [ ]:
result = result.drop(columns=['yearly_forecast', 'forecast_incl_NGC'])

In [ ]:
result = result.rename(columns={'yearly_forecast_new': 'yearly_forecast'})

In [ ]:
result = result.drop(columns=['yearly_budget'])

In [ ]:
result = result.rename(columns={'total_baseline': 'yearly_budget'})
result = result.rename(columns={'TOTAL Baseline Benefits (Łm)': 'total_benefits'})

In [ ]:
column_order = ['project_name','department','colour_rating','description_aims','rating_comment','start_date','end_date','schedule_comment','yearly_budget','yearly_forecast','wlc_baseline_incl_NCG','variance_comment','budget_comment','year','report_category','project_number','total_benefits','benefits_comment']

In [ ]:
result = result[column_order]

In [ ]:
df_macro = pd.read_csv('../data/macro/GDP.csv',encoding='iso-8859-2', delimiter=';')

In [ ]:
df_macro

In [ ]:
df_macro.info()

In [ ]:
# Schritt 1: Identifizieren der relevanten Zeilen
remained_scheduled_df = df_franzi[df_franzi['schedule_comment'].str.contains('remained scheduled', case=False)]

# Schritt 2: Aktuellsten 'end_date'-Wert für jedes 'project_name' ermitteln
latest_end_dates = df_franzi.groupby('project_name').apply(lambda x: x.loc[x['year'].idxmax(), 'end_date'])

# Schritt 3: Zuweisen des aktuellsten 'end_date' an die relevanten Zeilen
for index, row in remained_scheduled_df.iterrows():
    latest_end_date = latest_end_dates[row['project_name']]
    if pd.isna(row['end_date']) and pd.notna(latest_end_date):
        df_franzi.at[index, 'end_date'] = latest_end_date

# Ergebnis anzeigen
df_franzi


In [ ]:
# Schritt 2: Aktuellsten 'end_date'-Wert für jedes 'project_name' ermitteln
latest_end_dates = df_franzi.groupby('project_name').apply(lambda x: x.loc[x['year'].idxmax(), 'start_date'])

# Schritt 3: Zuweisen des aktuellsten 'end_date' an die relevanten Zeilen
for index, row in remained_scheduled_df.iterrows():
    latest_end_date = latest_end_dates[row['project_name']]
    if pd.isna(row['start_date']) and pd.notna(latest_end_date):
        df_franzi.at[index, 'start_date'] = latest_end_date

# Ergebnis anzeigen
df_franzi

In [ ]:
    month_mapping = {
        'jan': '01', 'january': '01',
        'feb': '02', 'february': '02',
        'mar': '03', 'march': '03',
        'apr': '04', 'april': '04',
        'may': '05', 'may': '05',
        'jun': '06', 'june': '06',
        'jul': '07', 'july': '07',
        'aug': '08', 'august': '08',
        'sep': '09', 'september': '09',
        'oct': '10', 'october': '10',
        'nov': '11', 'november': '11',
        'dec': '12', 'december': '12'
    }



In [ ]:
def extract_date_after_to_or_on(comment):
    REGEX = "(to|is|on)\s*(\d{1,2})\s+(\w+)\s+(\d{2,4})"
    x = re.compile(REGEX)
    
    match_REGEX = x.search(comment)      
    
    if match_REGEX:
        day = match_REGEX.group(2)
        month = match_REGEX.group(3)
        year = match_REGEX.group(4)
        if len(day) == 1:
            day = '0'+ day
        month = month_mapping.get(month.lower())
        if len(year) == 2:
            year = '20' + year
        return year +'-'+ month +'-'+ day
    
for index, row in df_franzi.iterrows():
    if pd.isna(row['end_date']):
        extracted_date = extract_date_after_to_or_on(row['schedule_comment'])
        if extracted_date:
            df_franzi.at[index, 'end_date'] = extracted_date

df_franzi



In [ ]:
# Der Code zum Filtern von Zeilen, in denen 'start_date' oder 'end_date' 'NaN' ist
# und die 'schedule_comment' nicht mit 'Exempt' oder '\r\nExempt' beginnen

nan_dates_df = df_franzi[
    ((df_franzi['end_date'].isna())) & 
    (~df_franzi['schedule_comment'].str.match(r'^(Exempt|\r\nExempt)'))
][['start_date', 'end_date', 'schedule_comment']]

# Ausgabe des gefilterten DataFrames
nan_dates_df

In [ ]:
# Der Code zum Filtern von Zeilen, in denen 'start_date' oder 'end_date' 'NaN' ist
# und die 'schedule_comment' nicht mit 'Exempt' oder '\r\nExempt' beginnen

nan_dates_df_start = df_franzi[
    ((df_franzi['start_date'].isna())) & 
    (~df_franzi['schedule_comment'].str.match(r'^(Exempt|\r\nExempt)'))
][['start_date', 'schedule_comment']]

# Ausgabe des gefilterten DataFrames
nan_dates_df_start

In [ ]:
df1 = pd.read_pickle("../data/pickle/cleaned_2021_2023.pkl")
df2 = pd.read_pickle('../data/pickle/cleaned_2018_2020.pkl')
df3 = pd.read_pickle('../data/pickle/cleaned_2014-2017.pkl')
df4 = pd.read_csv('../data/macro/GDP.csv',encoding='iso-8859-2', delimiter=';')

In [ ]:
df = pd.concat([df1, df2, df3], axis=0, ignore_index=True)

In [ ]:
df = df.merge(df4, on='year', how='left')

In [ ]:
df.drop(columns='Unnamed: 0')

In [ ]:
df["colour_rating"] = df["colour_rating"].str.lower()

# Drop rows where 'project_name' contains 'Unnamed:' followed by other characters
df = df[~df['project_name'].str.contains(r'Unnamed:.*', na=False)]

# Optionally, reset the index of the cleaned DataFrame
df = df.reset_index(drop=True)

## Imputing

In [ ]:
latest_start_dates = df.groupby('project_name').apply(lambda x: x.loc[x['year'].idxmax(), 'start_date'])

In [ ]:
for index, row in df.iterrows():
    latest_start_date = latest_start_dates[row['project_name']]
    if pd.isna(row['start_date']) and pd.notna(latest_start_date):
        df.at[index, 'start_date'] = latest_start_date

In [ ]:
latest_benefits = df.groupby('project_name').apply(lambda x: x.loc[x['year'].idxmax(), 'total_benefits'])

for index, row in df.iterrows():
    latest_benefit = latest_benefits[row['project_name']]
    if pd.isna(row['total_benefits']) and pd.notna(latest_benefit):
        df.at[index, 'total_benefits'] = latest_benefit

In [ ]:
latest_categories = df.groupby('project_name').apply(lambda x: x.loc[x['year'].idxmax(), 'report_category'])

for index, row in df.iterrows():
    latest_category = latest_categories[row['project_name']]
    if pd.isna(row['report_category']) and pd.notna(latest_category):
        df.at[index, 'report_category'] = latest_category

In [ ]:
latest_numbers = df.groupby('project_name').apply(lambda x: x.loc[x['year'].idxmax(), 'project_number'])

for index, row in df.iterrows():
    latest_number = latest_numbers[row['project_name']]
    if pd.isna(row['project_number']) and pd.notna(latest_number):
        df.at[index, 'project_number'] = latest_number

In [ ]:
df['description_aims'].unique().tolist()

In [ ]:
df_with_dummies = pd.get_dummies(df, columns=['department', 'colour_rating'], drop_first=True)


In [ ]:
df["start_date"] = pd.to_datetime(df["start_date"])
df["end_date"] = pd.to_datetime(df["end_date"])
# Calculate the duration in years
df["year_duration"] = (df["end_date"] - df["start_date"]).dt.days / 365.25

# Optionally, round the duration to a specific number of decimal places
df["year_duration"] = df["year_duration"].round(2)

In [ ]:
df['start_year'] = df['start_date'].dt.year
df['end_year'] = df['end_date'].dt.year

In [ ]:
macro_columns = pd.read_csv('../data/macro/GDP.csv', encoding='iso-8859-2', delimiter=';')

In [ ]:
df = df.drop(columns=['GDP', 'annual_earning_ft', 'unemployment_rate', 'youth_unemployment_rate', 'inflation_rate', 'population', 'gov_debt', 'tax_revenue', 'revenue_excl_grants', 'grants_and_other_revenue'])

In [ ]:
df = df.merge(macro_columns, on='year', how='left')

## Final Pickle

In [39]:
df = pd.read_pickle('../data/pickle/data_with_topics.pkl')

## Preprocessing

In [40]:
df = df.drop(columns=['description_aims', 'rating_comment', 'start_date',
       'end_date', 'schedule_comment', 'yearly_budget', 'yearly_forecast', 'variance_comment', 'budget_comment',
       'report_category', 'project_number', 'total_benefits',
       'benefits_comment', 'population', 'revenue_excl_grants', 'grants_and_other_revenue'])

In [41]:
df['year'].unique()

array([2023., 2022., 2021., 2018., 2019., 2020., 2014., 2015., 2016.,
       2017.])

In [42]:
df['year'] = df['year'].astype(int)

In [43]:
df = df.dropna(subset=['year'])

In [44]:
df = pd.get_dummies(df, columns=['topic', ])

In [45]:
# colour_rating_columns = [col for col in df.columns if col.startswith('colour_rating_')]
# department_columns = [col for col in df.columns if col.startswith('department_')]

# def latest_value(df, column):
#     try:
#         return df.loc[df['year'].idxmax(), column]
#     except Exception as e:
#         print(f"Error in Group {df.name} for column {column}: {e}")
#         return None

agg_dict = {
    'wlc_baseline_incl_NCG': 'max',
    'year_duration': 'max',
    'end_year': 'max',
    'start_year': 'min',
    'year': 'max',
    'GDP': 'mean',
    'annual_earning_ft': 'mean',
    'unemployment_rate': 'mean',
    'youth_unemployment_rate': 'mean',
    'inflation_rate': 'mean',
    'gov_debt': 'mean',
    'tax_revenue': 'mean',
    'department_CO': lambda x: x.mode()[0],
    'department_CPS': lambda x: x.mode()[0],
    'department_DBT': lambda x: x.mode()[0],
    'department_DCMS': lambda x: x.mode()[0],
    'department_DEFRA': lambda x: x.mode()[0],
    'department_DEFRA & DFT': lambda x: x.mode()[0],
    'department_DESNZ': lambda x: x.mode()[0],
    'department_DFE': lambda x: x.mode()[0],
    'department_DFID': lambda x: x.mode()[0],
    'department_DFT': lambda x: x.mode()[0],
    'department_DHSC': lambda x: x.mode()[0],
    'department_DLUHC': lambda x: x.mode()[0],
    'department_DSIT': lambda x: x.mode()[0],
    'department_DWP': lambda x: x.mode()[0],
    'department_FCDO': lambda x: x.mode()[0],
    'department_HMLR': lambda x: x.mode()[0],
    'department_HMRC': lambda x: x.mode()[0],
    'department_HMT': lambda x: x.mode()[0],
    'department_HO': lambda x: x.mode()[0],
    'department_MOD': lambda x: x.mode()[0],
    'department_MOJ': lambda x: x.mode()[0],
    'department_NCA': lambda x: x.mode()[0],
    'department_NS&I': lambda x: x.mode()[0],
    'department_ONS': lambda x: x.mode()[0],
    'department_VOA': lambda x: x.mode()[0],
    'topic_0': lambda x: x.mode()[0],
    'topic_1': lambda x: x.mode()[0],
    'topic_2': lambda x: x.mode()[0],
    'topic_3': lambda x: x.mode()[0],
    'topic_4': lambda x: x.mode()[0],
    'topic_5': lambda x: x.mode()[0],
    'topic_6': lambda x: x.mode()[0],
    'topic_7': lambda x: x.mode()[0],
    'topic_8': lambda x: x.mode()[0],
    'topic_9': lambda x: x.mode()[0],
    'topic_10': lambda x: x.mode()[0],
    'topic_11': lambda x: x.mode()[0],
    'topic_12': lambda x: x.mode()[0],
    'topic_13': lambda x: x.mode()[0],
    'topic_14': lambda x: x.mode()[0],
    'topic_15': lambda x: x.mode()[0],
    'topic_16': lambda x: x.mode()[0],
    'topic_17': lambda x: x.mode()[0],
    'topic_18': lambda x: x.mode()[0],
    'topic_19': lambda x: x.mode()[0],
    'colour_rating_amber/green': lambda x: x.mode()[0],
    'colour_rating_amber/red': lambda x: x.mode()[0],
    'colour_rating_green': lambda x: x.mode()[0],
    'colour_rating_red': lambda x: x.mode()[0],
    'colour_rating_reset': lambda x: x.mode()[0]
}

# for col in colour_rating_columns:
#     agg_dict[col] = lambda x, col=col: latest_value(x, col)

# for col in department_columns:
#     agg_dict[col] = lambda x, col=col: latest_value(x, col)

df = df.groupby('project_name').agg(agg_dict)


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 732 entries, 10,000 Additional Prison Places to care.data
Data columns (total 62 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   wlc_baseline_incl_NCG      674 non-null    float64
 1   year_duration              634 non-null    float64
 2   end_year                   698 non-null    float64
 3   start_year                 655 non-null    float64
 4   year                       732 non-null    int64  
 5   GDP                        732 non-null    float64
 6   annual_earning_ft          732 non-null    float64
 7   unemployment_rate          732 non-null    float64
 8   youth_unemployment_rate    732 non-null    float64
 9   inflation_rate             732 non-null    float64
 10  gov_debt                   732 non-null    float64
 11  tax_revenue                732 non-null    float64
 12  department_CO              732 non-null    bool   
 13  department_CPS     

In [47]:
df = df.drop(columns='year')

In [48]:
df = df.drop(columns=['year_duration', 'end_year', 'youth_unemployment_rate'])

In [49]:
df

,wlc_baseline_incl_NCG,start_year,GDP,annual_earning_ft,unemployment_rate,inflation_rate,gov_debt,tax_revenue,department_CO,department_CPS,...,topic_15,topic_16,topic_17,topic_18,topic_19,colour_rating_amber/green,colour_rating_amber/red,colour_rating_green,colour_rating_red,colour_rating_reset
project_name,,,,,,,,,,,,,,,,,,,,,
"10,000 Additional Prison Places",32614.00,NaN,2089346.0,31386.0,0.04650,0.028,42025705.0,5622525.0,False,False,...,False,False,False,False,False,False,False,False,False,False
"10,000 Additional Prison Places Programme - Estate Expansions",8333.00,NaN,2089346.0,31386.0,0.04650,0.028,42025705.0,5622525.0,False,False,...,False,False,False,False,False,False,False,False,False,False
"100,000 Genomes Project",350.00,2013.0,2091459.0,27887.0,0.05055,0.034,29771915.0,4933600.0,False,False,...,False,False,False,False,False,False,True,False,False,False
100000 Genomes Project,392.90,2013.0,2167415.0,28759.0,0.04330,0.027,33584200.0,5327300.0,False,False,...,False,False,False,False,False,False,False,False,False,False
10K Additional Prison Places Estate Expansion Category D,NaN,2020.0,2270764.0,33061.0,0.03730,0.092,35655860.0,6842110.0,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Youth Contract,741.93,2012.0,1963807.0,27011.0,0.07520,0.019,25231790.0,4474700.0,False,False,...,False,False,False,False,False,True,False,False,False,False
Youth Investment Fund,341.00,2022.0,2270764.0,33061.0,0.03730,0.092,35655860.0,6842110.0,False,False,...,False,False,False,False,False,False,False,False,False,False
Youth Justice Reform Programme,659.00,2016.0,2270764.0,33061.0,0.03730,0.092,35655860.0,6842110.0,False,False,...,False,False,False,False,False,False,False,False,False,False


## Modeling

In [51]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

df = df.dropna()

X = df.drop(columns=['wlc_baseline_incl_NCG'])
y = df['wlc_baseline_incl_NCG']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': model.coef_})
print(coefficients)


Mean Squared Error: 37075505.74756879
                      Feature   Coefficient
0                  start_year -2.934870e+02
1                         GDP  3.058649e-02
2           annual_earning_ft  9.793318e-01
3           unemployment_rate  1.641558e+05
4              inflation_rate  1.100330e+04
5                    gov_debt  2.781505e-05
6                 tax_revenue -3.452058e-03
7               department_CO -2.406879e+02
8              department_CPS  2.105782e+02
9              department_DBT -8.125203e+02
10            department_DCMS -1.187602e+03
11           department_DEFRA  1.222677e+03
12     department_DEFRA & DFT -2.058505e-08
13           department_DESNZ  8.161164e+03
14             department_DFE  6.791070e+02
15            department_DFID -1.075159e-08
16             department_DFT  4.008785e+03
17            department_DHSC -8.651311e+02
18           department_DLUHC  1.189878e+03
19            department_DSIT  1.515904e+03
20             department_DWP -1.12112

In [52]:
from xgboost import XGBRegressor

X = df.drop(columns=['wlc_baseline_incl_NCG'])
y = df['wlc_baseline_incl_NCG']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_reg = XGBRegressor(objective='reg:squarederror', random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

grid_search = GridSearchCV(estimator=xgb_reg, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, verbose=1)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f'Best Params: {best_params}')
print(f'Best negative MSE (Training): {best_score}')

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error (Test): {mse}')


Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Params: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 100, 'subsample': 0.8}
Best negative MSE (Training): -32994420.995696746
Mean Squared Error (Test): 37507910.14708265


In [53]:
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error: {rmse}')

Root Mean Squared Error: 6124.370183707272


In [54]:
from bayes_opt import BayesianOptimization
from hyperopt import hp, fmin, tpe

In [55]:
from sklearn.datasets import make_regression

X = df.drop(columns=['wlc_baseline_incl_NCG'])
y = df['wlc_baseline_incl_NCG']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function to minimize
def xgb_eval(max_depth, learning_rate, subsample, colsample_bytree):
    params = {
        'max_depth': int(max_depth),
        'learning_rate': learning_rate,
        'subsample': subsample,
        'colsample_bytree': colsample_bytree,
        'n_estimators': 100,
        'objective': 'reg:squarederror',
        'random_state': 42,
    }

    model = XGBRegressor(**params)
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    return score

# Define the search space
pbounds = {
    'max_depth': (3, 10),
    'learning_rate': (0.01, 0.3),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
}

# Perform Bayesian optimization
optimizer = BayesianOptimization(f=xgb_eval, pbounds=pbounds, random_state=42)
optimizer.maximize(init_points=5, n_iter=25)

# Print the best hyperparameters and score
print(f"Best hyperparameters: {optimizer.max['params']}")
print(f"Best score: {optimizer.max['target']:.4f}")

|   iter    |  target   | colsam... | learni... | max_depth | subsample |
-------------------------------------------------------------------------
| 1         | 0.1946    | 0.6873    | 0.2857    | 8.124     | 0.7993    |
| 2         | 0.1532    | 0.578     | 0.05524   | 3.407     | 0.9331    |
| 3         | 0.2566    | 0.8006    | 0.2153    | 3.144     | 0.985     |
| 4         | 0.369     | 0.9162    | 0.07158   | 4.273     | 0.5917    |
| 5         | 0.0759    | 0.6521    | 0.1622    | 6.024     | 0.6456    |
| 6         | 0.1961    | 0.6501    | 0.1943    | 3.755     | 0.985     |
| 7         | 0.1556    | 0.9673    | 0.2494    | 5.764     | 0.7293    |
| 8         | 0.01886   | 0.9827    | 0.1481    | 7.796     | 0.9787    |
| 9         | 0.1373    | 0.7604    | 0.1703    | 9.984     | 0.6671    |
| 10        | 0.3444    | 0.7845    | 0.1152    | 6.175     | 0.513     |
| 11        | 0.1292    | 0.9685    | 0.2277    | 5.028     | 0.9349    |
| 12        | 0.1134    | 0.6247    | 

In [56]:
X

,start_year,GDP,annual_earning_ft,unemployment_rate,inflation_rate,gov_debt,tax_revenue,department_CO,department_CPS,department_DBT,...,topic_15,topic_16,topic_17,topic_18,topic_19,colour_rating_amber/green,colour_rating_amber/red,colour_rating_green,colour_rating_red,colour_rating_reset
project_name,,,,,,,,,,,,,,,,,,,,,
"100,000 Genomes Project",2013.0,2091459.0,27887.0,0.05055,0.0340,29771915.0,4933600.0,False,False,False,...,False,False,False,False,False,False,True,False,False,False
100000 Genomes Project,2013.0,2167415.0,28759.0,0.04330,0.0270,33584200.0,5327300.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
16/17 New Property Model Programme,2015.0,2111357.0,28159.0,0.04810,0.0180,31190050.0,5066560.0,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1617 New Property Model Programme,2015.0,2182628.0,29159.0,0.04165,0.0235,33812850.0,5411150.0,True,False,False,...,False,False,False,False,False,False,False,False,False,False
18m+ Building Remediation Programme,2019.0,2270764.0,33061.0,0.03730,0.0920,35655860.0,6842110.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Youth Contract,2012.0,1963807.0,27011.0,0.07520,0.0190,25231790.0,4474700.0,False,False,False,...,False,False,False,False,False,True,False,False,False,False
Youth Investment Fund,2022.0,2270764.0,33061.0,0.03730,0.0920,35655860.0,6842110.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Youth Justice Reform Programme,2016.0,2270764.0,33061.0,0.03730,0.0920,35655860.0,6842110.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
